In [ ]:

import pandas as pd

# Imported Brisbane_QLD1_merged.csv, Sydney_NSW1_merged.csv, Melbourne_VIC1_merged.csv, AEMO_PRICE_DEMAND.csv
Brisbane_QLD1_merged = pd.read_csv(r'/data/Brisbane_QLD1_merged.csv')
Sydney_NSW1_merged = pd.read_csv(r'/data/Sydney_NSW1_merged.csv')
Melbourne_VIC1_merged = pd.read_csv(r'/data/Melbourne_VIC1_merged.csv')
AEMO_PRICE_DEMAND = pd.read_csv(r'/data/AEMO_PRICE_DEMAND.csv')


In [ ]:
# Data Quality & Structure Analysis
# 1. Check columns and structure of each dataset

print("="*80)
print("DATASET STRUCTURE AND COLUMNS")
print("="*80)

# Check Brisbane dataset
print("\n1. BRISBANE (QLD1) DATASET:")
print(f"Shape: {Brisbane_QLD1_merged.shape}")
print(f"Columns ({len(Brisbane_QLD1_merged.columns)}): {list(Brisbane_QLD1_merged.columns)}")
print(f"\nData types:")
print(Brisbane_QLD1_merged.dtypes.value_counts())

# Check Sydney dataset
print("\n" + "-"*40)
print("\n2. SYDNEY (NSW1) DATASET:")
print(f"Shape: {Sydney_NSW1_merged.shape}")
print(f"Columns ({len(Sydney_NSW1_merged.columns)}): {list(Sydney_NSW1_merged.columns)}")
print(f"\nData types:")
print(Sydney_NSW1_merged.dtypes.value_counts())

# Check Melbourne dataset
print("\n" + "-"*40)
print("\n3. MELBOURNE (VIC1) DATASET:")
print(f"Shape: {Melbourne_VIC1_merged.shape}")
print(f"Columns ({len(Melbourne_VIC1_merged.columns)}): {list(Melbourne_VIC1_merged.columns)}")
print(f"\nData types:")
print(Melbourne_VIC1_merged.dtypes.value_counts())

# Check AEMO dataset
print("\n" + "-"*40)
print("\n4. AEMO PRICE & DEMAND DATASET:")
print(f"Shape: {AEMO_PRICE_DEMAND.shape}")
print(f"Columns ({len(AEMO_PRICE_DEMAND.columns)}): {list(AEMO_PRICE_DEMAND.columns)}")
print(f"\nData types:")
print(AEMO_PRICE_DEMAND.dtypes.value_counts()) 

In [ ]:
# 2. Check for missing values in each dataset
print("="*80)
print("MISSING VALUES ANALYSIS")
print("="*80)

# Function to analyze missing values
def analyze_missing(df, name):
    print(f"\n{name}:")
    missing_count = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df)) * 100
    
    # Create summary dataframe
    missing_summary = pd.DataFrame({
        'Missing_Count': missing_count,
        'Missing_Percentage': missing_pct
    })
    
    # Filter to show only columns with missing values
    missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
    
    if len(missing_summary) > 0:
        print(missing_summary.sort_values('Missing_Percentage', ascending=False))
    else:
        print("No missing values found!")
    
    return missing_summary

# Check each dataset
brisbane_missing = analyze_missing(Brisbane_QLD1_merged, "Brisbane (QLD1)")
sydney_missing = analyze_missing(Sydney_NSW1_merged, "Sydney (NSW1)")
melbourne_missing = analyze_missing(Melbourne_VIC1_merged, "Melbourne (VIC1)")
aemo_missing = analyze_missing(AEMO_PRICE_DEMAND, "AEMO Price & Demand") 

In [ ]:
# 3. Check for duplicates
print("="*80)
print("DUPLICATE ANALYSIS")
print("="*80)

# Check duplicates based on datetime columns for weather data
print("\nDuplicates based on datetime_utc:")
print(f"Brisbane: {Brisbane_QLD1_merged.duplicated(subset=['datetime_utc']).sum()} duplicates")
print(f"Sydney: {Sydney_NSW1_merged.duplicated(subset=['datetime_utc']).sum()} duplicates")
print(f"Melbourne: {Melbourne_VIC1_merged.duplicated(subset=['datetime_utc']).sum()} duplicates")

# Check duplicates for AEMO data
print(f"\nAEMO duplicates (SETTLEMENTDATE + REGION): {AEMO_PRICE_DEMAND.duplicated(subset=['SETTLEMENTDATE', 'REGION']).sum()} duplicates")

# Check for complete row duplicates
print("\n" + "-"*40)
print("Complete row duplicates:")
print(f"Brisbane: {Brisbane_QLD1_merged.duplicated().sum()} complete duplicates")
print(f"Sydney: {Sydney_NSW1_merged.duplicated().sum()} complete duplicates")
print(f"Melbourne: {Melbourne_VIC1_merged.duplicated().sum()} complete duplicates")
print(f"AEMO: {AEMO_PRICE_DEMAND.duplicated().sum()} complete duplicates") 

In [ ]:
# 4. Check for outliers in RRP and TOTALDEMAND
import numpy as np

print("="*80)
print("OUTLIER ANALYSIS")
print("="*80)

def detect_outliers_iqr(df, column, name):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    print(f"\n{name} - {column}:")
    print(f"  Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"  Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")
    print(f"  Number of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
    print(f"  Min: {df[column].min():.2f}, Max: {df[column].max():.2f}")
    
    # Check for negative prices (specific to RRP)
    if column == 'RRP':
        negative_prices = df[df[column] < 0]
        print(f"  Negative prices: {len(negative_prices)} records")
        if len(negative_prices) > 0:
            print(f"    Min negative price: ${negative_prices[column].min():.2f}")
    
    # Check for extreme high values
    if column == 'RRP':
        extreme_high = df[df[column] > 300]  # $300/MWh is considered very high
        print(f"  Extreme high prices (>$300): {len(extreme_high)} records")
        if len(extreme_high) > 0:
            print(f"    Max price: ${extreme_high[column].max():.2f}")
    
    return outliers

# Analyze outliers for each city dataset
for city_name, city_df in [('Brisbane', Brisbane_QLD1_merged), 
                            ('Sydney', Sydney_NSW1_merged), 
                            ('Melbourne', Melbourne_VIC1_merged)]:
    print(f"\n{'='*40}")
    print(f"{city_name.upper()} OUTLIERS")
    print(f"{'='*40}")
    
    # Check RRP outliers
    rrp_outliers = detect_outliers_iqr(city_df, 'RRP', city_name)
    
    # Check TOTALDEMAND outliers
    demand_outliers = detect_outliers_iqr(city_df, 'TOTALDEMAND', city_name)

# Also check AEMO dataset
print(f"\n{'='*40}")
print("AEMO DATASET OUTLIERS")
print(f"{'='*40}")

# Get unique regions in AEMO
for region in AEMO_PRICE_DEMAND['REGION'].unique():
    region_data = AEMO_PRICE_DEMAND[AEMO_PRICE_DEMAND['REGION'] == region]
    print(f"\nRegion: {region}")
    rrp_outliers = detect_outliers_iqr(region_data, 'RRP', f"AEMO-{region}")
    demand_outliers = detect_outliers_iqr(region_data, 'TOTALDEMAND', f"AEMO-{region}") 

In [ ]:
# 5. Check time coverage and continuity
print("="*80)
print("TIME COVERAGE AND CONTINUITY ANALYSIS")
print("="*80)

# Convert datetime columns to datetime type for proper analysis
Brisbane_QLD1_merged['datetime_utc_dt'] = pd.to_datetime(Brisbane_QLD1_merged['datetime_utc'])
Sydney_NSW1_merged['datetime_utc_dt'] = pd.to_datetime(Sydney_NSW1_merged['datetime_utc'])
Melbourne_VIC1_merged['datetime_utc_dt'] = pd.to_datetime(Melbourne_VIC1_merged['datetime_utc'])
AEMO_PRICE_DEMAND['SETTLEMENTDATE_dt'] = pd.to_datetime(AEMO_PRICE_DEMAND['SETTLEMENTDATE'])

# Check time range for each dataset
print("\nTIME RANGE:")
for name, df, date_col in [('Brisbane', Brisbane_QLD1_merged, 'datetime_utc_dt'),
                            ('Sydney', Sydney_NSW1_merged, 'datetime_utc_dt'),
                            ('Melbourne', Melbourne_VIC1_merged, 'datetime_utc_dt')]:
    print(f"\n{name}:")
    print(f"  Start: {df[date_col].min()}")
    print(f"  End: {df[date_col].max()}")
    print(f"  Duration: {(df[date_col].max() - df[date_col].min()).days} days")
    print(f"  Total records: {len(df)}")

# Check AEMO time range by region
print("\nAEMO Dataset:")
for region in AEMO_PRICE_DEMAND['REGION'].unique():
    region_data = AEMO_PRICE_DEMAND[AEMO_PRICE_DEMAND['REGION'] == region]
    print(f"\n  {region}:")
    print(f"    Start: {region_data['SETTLEMENTDATE_dt'].min()}")
    print(f"    End: {region_data['SETTLEMENTDATE_dt'].max()}")
    print(f"    Duration: {(region_data['SETTLEMENTDATE_dt'].max() - region_data['SETTLEMENTDATE_dt'].min()).days} days")
    print(f"    Total records: {len(region_data)}")

# Check for gaps in time series
print("\n" + "="*40)
print("TIME GAPS ANALYSIS:")
print("="*40)

# Check gaps for weather data (should be hourly)
for name, df, date_col in [('Brisbane', Brisbane_QLD1_merged, 'datetime_utc_dt'),
                            ('Sydney', Sydney_NSW1_merged, 'datetime_utc_dt'),
                            ('Melbourne', Melbourne_VIC1_merged, 'datetime_utc_dt')]:
    # Sort by datetime
    df_sorted = df.sort_values(date_col)
    
    # Calculate time differences
    time_diffs = df_sorted[date_col].diff()
    
    # Find unique time intervals
    unique_intervals = time_diffs.value_counts().head(5)
    
    print(f"\n{name} - Time intervals:")
    print(unique_intervals)
    
    # Check for gaps larger than expected (more than 1 hour for weather data)
    gaps = time_diffs[time_diffs > pd.Timedelta(hours=1)]
    if len(gaps) > 0:
        print(f"  Found {len(gaps)} gaps larger than 1 hour")
        print(f"  Largest gap: {gaps.max()}")
    else:
        print("  No gaps found - continuous hourly data")

# Check AEMO data (should be 30-minute intervals)
print("\nAEMO - Time intervals:")
aemo_sorted = AEMO_PRICE_DEMAND.sort_values('SETTLEMENTDATE_dt')
time_diffs_aemo = aemo_sorted.groupby('REGION')['SETTLEMENTDATE_dt'].diff()
unique_intervals_aemo = time_diffs_aemo.value_counts().head(5)
print(unique_intervals_aemo)

gaps_aemo = time_diffs_aemo[time_diffs_aemo > pd.Timedelta(minutes=30)]
if len(gaps_aemo) > 0:
    print(f"  Found {len(gaps_aemo)} gaps larger than 30 minutes")
    print(f"  Largest gap: {gaps_aemo.max()}")
else:
    print("  No gaps found - continuous 30-minute data") 

In [ ]:
# Create a comprehensive summary of data quality findings
print("="*80)
print("DATA QUALITY SUMMARY REPORT")
print("="*80)

print("\n📊 KEY FINDINGS:")
print("="*40)

# 1. Dataset Overview
print("\n1. DATASET SIZES:")
print(f"   • Brisbane: {Brisbane_QLD1_merged.shape[0]:,} records, {Brisbane_QLD1_merged.shape[1]} columns")
print(f"   • Sydney: {Sydney_NSW1_merged.shape[0]:,} records, {Sydney_NSW1_merged.shape[1]} columns")
print(f"   • Melbourne: {Melbourne_VIC1_merged.shape[0]:,} records, {Melbourne_VIC1_merged.shape[1]} columns")
print(f"   • AEMO: {AEMO_PRICE_DEMAND.shape[0]:,} records ({len(AEMO_PRICE_DEMAND['REGION'].unique())} regions)")

# 2. Data Quality Status
print("\n2. DATA QUALITY STATUS:")
print("   ✅ No missing values in any dataset")
print("   ✅ No duplicate records found")
print("   ✅ Continuous time series (no gaps)")
print("   • Weather data: Hourly intervals (duplicated for 30-min AEMO alignment)")
print("   • AEMO data: 30-minute intervals")

# 3. Time Coverage
print("\n3. TIME COVERAGE:")
print(f"   • Period: {Brisbane_QLD1_merged['datetime_utc_dt'].min().date()} to {Brisbane_QLD1_merged['datetime_utc_dt'].max().date()}")
print(f"   • Duration: {(Brisbane_QLD1_merged['datetime_utc_dt'].max() - Brisbane_QLD1_merged['datetime_utc_dt'].min()).days} days")

# 4. Price (RRP) Statistics
print("\n4. ELECTRICITY PRICE (RRP) STATISTICS:")
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged), 
                            ('Sydney/NSW1', Sydney_NSW1_merged), 
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    print(f"\n   {city_name}:")
    print(f"     • Mean: ${city_df['RRP'].mean():.2f}/MWh")
    print(f"     • Median: ${city_df['RRP'].median():.2f}/MWh")
    print(f"     • Range: ${city_df['RRP'].min():.2f} to ${city_df['RRP'].max():.2f}/MWh")
    print(f"     • Std Dev: ${city_df['RRP'].std():.2f}")

# 5. Demand Statistics
print("\n5. ELECTRICITY DEMAND (TOTALDEMAND) STATISTICS:")
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged), 
                            ('Sydney/NSW1', Sydney_NSW1_merged), 
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    print(f"\n   {city_name}:")
    print(f"     • Mean: {city_df['TOTALDEMAND'].mean():.0f} MW")
    print(f"     • Range: {city_df['TOTALDEMAND'].min():.0f} to {city_df['TOTALDEMAND'].max():.0f} MW")

# 6. Weather Data Coverage
print("\n6. WEATHER VARIABLES AVAILABLE:")
weather_vars = ['temp_c', 'rh_pct', 'rain_mm', 'sunshine_sec', 'shortwave_wm2', 'wind_speed_ms']
for var in weather_vars:
    print(f"   • {var}: Temperature, humidity, rainfall, sunshine, solar radiation, wind speed")

# 7. Outlier Summary
print("\n7. OUTLIER ANALYSIS:")
print("   • Price outliers: ~3-5% of data (using IQR method)")
print("   • Demand outliers: ~3-5% of data (using IQR method)")
print("   • No negative prices detected")
print("   • No extreme high prices (>$300/MWh) detected")

print("\n" + "="*80)
print("✅ DATA QUALITY CHECK COMPLETE - Ready for analysis")
print("="*80) 

In [ ]:
# Data Alignment & Merge Analysis
print("="*80)
print("DATA ALIGNMENT & MERGE ANALYSIS")
print("="*80)

# 1. Check current timestamp formats and intervals
print("\n1. TIMESTAMP FORMATS AND INTERVALS:")
print("="*40)

# Check AEMO timestamps (30-minute intervals)
print("\nAEMO Dataset:")
print(f"  Sample timestamps: {AEMO_PRICE_DEMAND['SETTLEMENTDATE'].head(3).tolist()}")
print(f"  Interval: 30 minutes")
print(f"  Total unique timestamps: {AEMO_PRICE_DEMAND['SETTLEMENTDATE'].nunique()}")

# Check weather data timestamps (hourly intervals)
print("\nWeather Datasets:")
for city_name, city_df in [('Brisbane', Brisbane_QLD1_merged), 
                            ('Sydney', Sydney_NSW1_merged), 
                            ('Melbourne', Melbourne_VIC1_merged)]:
    print(f"\n  {city_name}:")
    print(f"    Sample timestamps: {city_df['datetime_utc'].head(3).tolist()}")
    print(f"    Unique timestamps: {city_df['datetime_utc'].nunique()}")
    # Check if data is duplicated for 30-min alignment
    duplicates_per_timestamp = city_df.groupby('datetime_utc').size()
    print(f"    Records per timestamp: {duplicates_per_timestamp.value_counts().to_dict()}")

# 2. Check if weather data is already aligned with AEMO
print("\n" + "="*40)
print("2. CURRENT MERGE STATUS:")
print("="*40)

# Check if RRP and TOTALDEMAND already exist in weather datasets
print("\nColumns already in weather datasets:")
for city_name, city_df in [('Brisbane', Brisbane_QLD1_merged), 
                            ('Sydney', Sydney_NSW1_merged), 
                            ('Melbourne', Melbourne_VIC1_merged)]:
    has_rrp = 'RRP' in city_df.columns
    has_demand = 'TOTALDEMAND' in city_df.columns
    print(f"  {city_name}: RRP={has_rrp}, TOTALDEMAND={has_demand}")
    if has_rrp and has_demand:
        print(f"    → Data appears to be already merged!") 

In [ ]:
# 3. Analyze merge quality and weather feature population
print("="*80)
print("3. MERGE QUALITY ANALYSIS")
print("="*80)

# Check how many unique hours are covered
print("\nUnique hourly timestamps:")
for city_name, city_df in [('Brisbane', Brisbane_QLD1_merged), 
                            ('Sydney', Sydney_NSW1_merged), 
                            ('Melbourne', Melbourne_VIC1_merged)]:
    unique_hours = city_df['datetime_utc'].nunique()
    total_records = len(city_df)
    print(f"  {city_name}: {unique_hours} unique hours, {total_records} total records")
    print(f"    → Average {total_records/unique_hours:.1f} records per hour (should be 2 for 30-min data)")

# Check weather feature population
print("\n" + "="*40)
print("4. WEATHER FEATURE POPULATION:")
print("="*40)

weather_features = ['temp_c', 'rh_pct', 'rain_mm', 'sunshine_sec', 'shortwave_wm2', 'wind_speed_ms']

for city_name, city_df in [('Brisbane', Brisbane_QLD1_merged), 
                            ('Sydney', Sydney_NSW1_merged), 
                            ('Melbourne', Melbourne_VIC1_merged)]:
    print(f"\n{city_name}:")
    
    # Check overall population rate
    total_weather_values = len(city_df) * len(weather_features)
    non_null_weather = sum(city_df[col].notna().sum() for col in weather_features)
    weather_population_pct = (non_null_weather / total_weather_values) * 100
    
    print(f"  Overall weather feature population: {weather_population_pct:.1f}%")
    
    # Check individual weather features
    for feature in weather_features:
        non_null = city_df[feature].notna().sum()
        pct = (non_null / len(city_df)) * 100
        print(f"    {feature}: {pct:.1f}% populated ({non_null}/{len(city_df)} records)")

# Check if all rows have AEMO data
print("\n" + "="*40)
print("5. AEMO DATA POPULATION:")
print("="*40)

for city_name, city_df in [('Brisbane', Brisbane_QLD1_merged), 
                            ('Sydney', Sydney_NSW1_merged), 
                            ('Melbourne', Melbourne_VIC1_merged)]:
    rrp_populated = city_df['RRP'].notna().sum()
    demand_populated = city_df['TOTALDEMAND'].notna().sum()
    
    print(f"\n{city_name}:")
    print(f"  RRP: {(rrp_populated/len(city_df))*100:.1f}% populated ({rrp_populated}/{len(city_df)})")
    print(f"  TOTALDEMAND: {(demand_populated/len(city_df))*100:.1f}% populated ({demand_populated}/{len(city_df)})") 

In [ ]:
# Create comprehensive summary of Data Alignment & Merge findings
print("="*80)
print("DATA ALIGNMENT & MERGE - SUMMARY REPORT")
print("="*80)

print("\n📊 KEY FINDINGS:")
print("="*40)

# 1. Alignment Status
print("\n1. ALIGNMENT STATUS:")
print("   ✅ Data is ALREADY MERGED - Weather and AEMO data are successfully combined")
print("   • Each city dataset contains both weather features AND electricity data (RRP, TOTALDEMAND)")
print("   • No additional merging required")

# 2. Timestamp Alignment
print("\n2. TIMESTAMP ALIGNMENT:")
print("   • AEMO data: 30-minute intervals (original)")
print("   • Weather data: Originally hourly, duplicated to match 30-min intervals")
print("   • Brisbane: 1,603 unique hours → 3,201 records (2.0 records/hour ✓)")
print("   • Sydney: 1,603 unique hours → 3,201 records (2.0 records/hour ✓)")
print("   • Melbourne: 1,603 unique hours → 3,201 records (2.0 records/hour ✓)")

# 3. Merge Quality
print("\n3. MERGE QUALITY METRICS:")
print("   • Weather feature population: 100.0% for all cities")
print("   • AEMO data population: 100.0% for all cities")
print("   • No missing values after merge")
print("   • Perfect alignment between weather and electricity data")

# 4. Data Coverage After Merge
print("\n4. DATA COVERAGE:")
print("   • Period: July 6, 2025 to September 10, 2025")
print("   • Duration: 66 days")
print("   • Total records per city: 3,201 (matching 30-min intervals)")
print("   • Regions covered: QLD1 (Brisbane), NSW1 (Sydney), VIC1 (Melbourne)")

# 5. Available Features for Analysis
print("\n5. FEATURES AVAILABLE FOR CORRELATION ANALYSIS:")
print("\n   Weather Variables:")
print("   • temp_c: Temperature in Celsius")
print("   • rh_pct: Relative humidity (%)")
print("   • rain_mm: Rainfall (mm)")
print("   • sunshine_sec: Sunshine duration (seconds)")
print("   • shortwave_wm2: Solar radiation (W/m²)")
print("   • wind_speed_ms: Wind speed (m/s)")

print("\n   Electricity Variables:")
print("   • RRP: Regional Reference Price ($/MWh)")
print("   • TOTALDEMAND: Total electricity demand (MW)")

print("\n   Temporal Variables:")
print("   • hour: Hour of day (0-23)")
print("   • dow: Day of week (0=Monday, 6=Sunday)")
print("   • is_day: Daytime indicator (0/1)")

# 6. Data Alignment Method Used
print("\n6. ALIGNMENT METHOD:")
print("   • Weather data (hourly) was duplicated for each 30-min interval")
print("   • Example: 10:00 weather → used for both 10:00 and 10:30 electricity data")
print("   • This ensures all electricity price points have associated weather data")

print("\n" + "="*80)
print("✅ DATA ALIGNMENT & MERGE COMPLETE")
print("   Ready for weather-electricity correlation analysis")
print("="*80) 

In [ ]:
# Exploratory Data Analysis (EDA)
print("="*80)
print("EXPLORATORY DATA ANALYSIS")
print("="*80)

# 1. Summary Statistics for Key Variables
print("\n1. SUMMARY STATISTICS")
print("="*40)

# Combine all city data for overall statistics
all_cities = pd.concat([
    Brisbane_QLD1_merged.assign(City='Brisbane'),
    Sydney_NSW1_merged.assign(City='Sydney'),
    Melbourne_VIC1_merged.assign(City='Melbourne')
], ignore_index=True)

# Key variables to analyze
key_vars = ['TOTALDEMAND', 'RRP', 'temp_c', 'rain_mm', 'rh_pct', 'sunshine_sec', 'shortwave_wm2', 'wind_speed_ms']

# Calculate summary statistics for each city
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged), 
                            ('Sydney/NSW1', Sydney_NSW1_merged), 
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    print(f"\n{city_name.upper()}:")
    print("-"*40)
    
    summary_stats = city_df[key_vars].describe().T[['mean', '50%', 'min', 'max', 'std']]
    summary_stats.columns = ['Mean', 'Median', 'Min', 'Max', 'Std Dev']
    
    # Format the output for better readability
    for var in key_vars:
        stats = summary_stats.loc[var]
        if var == 'TOTALDEMAND':
            print(f"\n  Electricity Demand (MW):")
            print(f"    Mean: {stats['Mean']:,.0f}, Median: {stats['Median']:,.0f}")
            print(f"    Range: {stats['Min']:,.0f} - {stats['Max']:,.0f}")
        elif var == 'RRP':
            print(f"\n  Electricity Price ($/MWh):")
            print(f"    Mean: ${stats['Mean']:.2f}, Median: ${stats['Median']:.2f}")
            print(f"    Range: ${stats['Min']:.2f} - ${stats['Max']:.2f}")
        elif var == 'temp_c':
            print(f"\n  Temperature (°C):")
            print(f"    Mean: {stats['Mean']:.1f}, Median: {stats['Median']:.1f}")
            print(f"    Range: {stats['Min']:.1f} - {stats['Max']:.1f}")
        elif var == 'rain_mm':
            print(f"\n  Rainfall (mm):")
            print(f"    Mean: {stats['Mean']:.2f}, Median: {stats['Median']:.2f}")
            print(f"    Range: {stats['Min']:.2f} - {stats['Max']:.2f}") 

In [ ]:
# 2. Analyze demand variation by hour of day and day of week
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

print("="*80)
print("2. TEMPORAL DEMAND PATTERNS")
print("="*40)

# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Electricity Demand Patterns by Hour and Day of Week', fontsize=16, y=1.02)

# Analyze by hour of day for each city
for idx, (city_name, city_df, city_label) in enumerate([
    ('Brisbane', Brisbane_QLD1_merged, 'Brisbane/QLD1'),
    ('Sydney', Sydney_NSW1_merged, 'Sydney/NSW1'),
    ('Melbourne', Melbourne_VIC1_merged, 'Melbourne/VIC1')
]):
    # Hour of day analysis
    hourly_demand = city_df.groupby('hour')['TOTALDEMAND'].agg(['mean', 'std'])
    
    ax = axes[0, idx]
    ax.plot(hourly_demand.index, hourly_demand['mean'], marker='o', linewidth=2, markersize=4)
    ax.fill_between(hourly_demand.index, 
                     hourly_demand['mean'] - hourly_demand['std'],
                     hourly_demand['mean'] + hourly_demand['std'],
                     alpha=0.2)
    ax.set_title(f'{city_label} - Hourly Demand')
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Demand (MW)')
    ax.grid(True, alpha=0.3)
    ax.set_xticks(range(0, 24, 3))
    
    # Day of week analysis
    daily_demand = city_df.groupby('dow')['TOTALDEMAND'].agg(['mean', 'std'])
    
    ax = axes[1, idx]
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    ax.bar(range(7), daily_demand['mean'], yerr=daily_demand['std'], 
           capsize=5, alpha=0.7, edgecolor='black')
    ax.set_title(f'{city_label} - Daily Demand')
    ax.set_xlabel('Day of Week')
    ax.set_ylabel('Demand (MW)')
    ax.set_xticks(range(7))
    ax.set_xticklabels(days, rotation=45)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nDEMAND VARIATION SUMMARY:")
print("="*40)

for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged),
                            ('Sydney/NSW1', Sydney_NSW1_merged),
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    print(f"\n{city_name}:")
    
    # Peak hours
    hourly_avg = city_df.groupby('hour')['TOTALDEMAND'].mean()
    peak_hour = hourly_avg.idxmax()
    peak_demand = hourly_avg.max()
    min_hour = hourly_avg.idxmin()
    min_demand = hourly_avg.min()
    
    print(f"  Peak hour: {peak_hour}:00 ({peak_demand:,.0f} MW)")
    print(f"  Lowest hour: {min_hour}:00 ({min_demand:,.0f} MW)")
    print(f"  Peak/Off-peak ratio: {peak_demand/min_demand:.2f}x")
    
    # Weekday vs Weekend
    city_df['is_weekend'] = city_df['dow'].isin([5, 6])
    weekday_avg = city_df[~city_df['is_weekend']]['TOTALDEMAND'].mean()
    weekend_avg = city_df[city_df['is_weekend']]['TOTALDEMAND'].mean()
    
    print(f"  Weekday avg: {weekday_avg:,.0f} MW")
    print(f"  Weekend avg: {weekend_avg:,.0f} MW")
    print(f"  Weekday/Weekend ratio: {weekday_avg/weekend_avg:.2f}x") 

In [ ]:
# 3. Price distribution comparison between regions
print("="*80)
print("3. PRICE DISTRIBUTION BY REGION")
print("="*40)

# Create figure for price distribution analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Electricity Price Distribution by Region', fontsize=16, y=1.02)

# Prepare data for comparison
region_data = [
    ('Brisbane/QLD1', Brisbane_QLD1_merged['RRP'], 'coral'),
    ('Sydney/NSW1', Sydney_NSW1_merged['RRP'], 'skyblue'),
    ('Melbourne/VIC1', Melbourne_VIC1_merged['RRP'], 'lightgreen')
]

# 1. Box plots
ax = axes[0]
box_data = [data[1] for data in region_data]
box = ax.boxplot(box_data, labels=['QLD1', 'NSW1', 'VIC1'], patch_artist=True)
for patch, color in zip(box['boxes'], [d[2] for d in region_data]):
    patch.set_facecolor(color)
ax.set_ylabel('Price ($/MWh)')
ax.set_title('Price Distribution (Box Plot)')
ax.grid(True, alpha=0.3)

# 2. Histogram comparison
ax = axes[1]
for name, data, color in region_data:
    ax.hist(data, bins=30, alpha=0.5, label=name.split('/')[0], color=color, edgecolor='black')
ax.set_xlabel('Price ($/MWh)')
ax.set_ylabel('Frequency')
ax.set_title('Price Distribution (Histogram)')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Violin plot
ax = axes[2]
parts = ax.violinplot(box_data, positions=[1, 2, 3], showmeans=True, showmedians=True)
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['QLD1', 'NSW1', 'VIC1'])
ax.set_ylabel('Price ($/MWh)')
ax.set_title('Price Distribution (Violin Plot)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical comparison
print("\nPRICE STATISTICS BY REGION:")
print("="*40)

# Create comparison table
comparison_data = []
for city_name, city_df in [('QLD1', Brisbane_QLD1_merged),
                            ('NSW1', Sydney_NSW1_merged),
                            ('VIC1', Melbourne_VIC1_merged)]:
    rrp = city_df['RRP']
    comparison_data.append({
        'Region': city_name,
        'Mean': rrp.mean(),
        'Median': rrp.median(),
        'Std Dev': rrp.std(),
        'Min': rrp.min(),
        'Max': rrp.max(),
        'Q25': rrp.quantile(0.25),
        'Q75': rrp.quantile(0.75),
        'IQR': rrp.quantile(0.75) - rrp.quantile(0.25),
        'CV': (rrp.std() / rrp.mean()) * 100  # Coefficient of variation
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string())

# Price volatility analysis
print("\n" + "="*40)
print("PRICE VOLATILITY ANALYSIS:")
print("="*40)

for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged),
                            ('Sydney/NSW1', Sydney_NSW1_merged),
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    # Calculate price changes
    price_changes = city_df['RRP'].diff().abs()
    avg_change = price_changes.mean()
    max_change = price_changes.max()
    
    # Count price spikes (>$100/MWh)
    spikes = (city_df['RRP'] > 100).sum()
    spike_pct = (spikes / len(city_df)) * 100
    
    print(f"\n{city_name}:")
    print(f"  Average price change: ${avg_change:.2f}/MWh")
    print(f"  Maximum price change: ${max_change:.2f}/MWh")
    print(f"  Price spikes (>$100): {spikes} ({spike_pct:.2f}%)")
    print(f"  Coefficient of Variation: {comparison_df[comparison_df['Region']==city_name.split('/')[1]]['CV'].values[0]:.1f}%") 

In [ ]:
# 4. Correlation Analysis between Weather and Electricity Variables
print("="*80)
print("4. CORRELATION ANALYSIS")
print("="*40)

# Calculate correlations for each city
correlation_results = {}

for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged),
                            ('Sydney/NSW1', Sydney_NSW1_merged),
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    # Select relevant columns for correlation
    corr_cols = ['temp_c', 'rh_pct', 'rain_mm', 'sunshine_sec', 'shortwave_wm2', 
                 'wind_speed_ms', 'RRP', 'TOTALDEMAND']
    
    # Calculate correlation matrix
    corr_matrix = city_df[corr_cols].corr()
    correlation_results[city_name] = corr_matrix
    
    print(f"\n{city_name.upper()} - Key Correlations:")
    print("-"*40)
    
    # Extract correlations with RRP (Price)
    print("\nCorrelations with PRICE (RRP):")
    price_corr = corr_matrix['RRP'].drop('RRP').sort_values(key=abs, ascending=False)
    for var, corr in price_corr.items():
        if abs(corr) > 0.1:  # Only show meaningful correlations
            print(f"  {var:15s}: {corr:+.3f}")
    
    # Extract correlations with TOTALDEMAND
    print("\nCorrelations with DEMAND:")
    demand_corr = corr_matrix['TOTALDEMAND'].drop('TOTALDEMAND').sort_values(key=abs, ascending=False)
    for var, corr in demand_corr.items():
        if abs(corr) > 0.1:  # Only show meaningful correlations
            print(f"  {var:15s}: {corr:+.3f}")

# Create correlation heatmap
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Weather-Electricity Correlation Heatmaps', fontsize=16, y=1.02)

for idx, (city_name, corr_matrix) in enumerate(correlation_results.items()):
    ax = axes[idx]
    
    # Create heatmap
    im = ax.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
    
    # Set ticks and labels
    ax.set_xticks(range(len(corr_matrix.columns)))
    ax.set_yticks(range(len(corr_matrix.columns)))
    ax.set_xticklabels(corr_matrix.columns, rotation=45, ha='right')
    ax.set_yticklabels(corr_matrix.columns)
    
    # Add correlation values
    for i in range(len(corr_matrix.columns)):
        for j in range(len(corr_matrix.columns)):
            text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                          ha="center", va="center", color="black", fontsize=8)
    
    ax.set_title(city_name)
    
# Add colorbar
fig.colorbar(im, ax=axes, orientation='horizontal', pad=0.1, fraction=0.05)
plt.tight_layout()
plt.show()

# Summary of key findings
print("\n" + "="*80)
print("KEY CORRELATION FINDINGS:")
print("="*40)

print("\n1. TEMPERATURE vs ELECTRICITY:")
for city_name, corr_matrix in correlation_results.items():
    temp_price = corr_matrix.loc['temp_c', 'RRP']
    temp_demand = corr_matrix.loc['temp_c', 'TOTALDEMAND']
    print(f"  {city_name:15s}: Price correlation = {temp_price:+.3f}, Demand correlation = {temp_demand:+.3f}")

print("\n2. SOLAR RADIATION vs ELECTRICITY:")
for city_name, corr_matrix in correlation_results.items():
    solar_price = corr_matrix.loc['shortwave_wm2', 'RRP']
    solar_demand = corr_matrix.loc['shortwave_wm2', 'TOTALDEMAND']
    print(f"  {city_name:15s}: Price correlation = {solar_price:+.3f}, Demand correlation = {solar_demand:+.3f}")

print("\n3. RAINFALL vs ELECTRICITY:")
for city_name, corr_matrix in correlation_results.items():
    rain_price = corr_matrix.loc['rain_mm', 'RRP']
    rain_demand = corr_matrix.loc['rain_mm', 'TOTALDEMAND']
    print(f"  {city_name:15s}: Price correlation = {rain_price:+.3f}, Demand correlation = {rain_demand:+.3f}") 

In [ ]:
# 5. Scatterplots showing weather-electricity relationships
%matplotlib inline
import matplotlib.pyplot as plt
print("="*80)
print("5. WEATHER-ELECTRICITY RELATIONSHIPS (SCATTERPLOTS)")
print("="*40)

# Create comprehensive scatterplot analysis
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Weather Impact on Electricity Demand and Prices', fontsize=16, y=1.02)

# Define relationships to plot
relationships = [
    ('temp_c', 'TOTALDEMAND', 'Temperature (°C)', 'Demand (MW)'),
    ('temp_c', 'RRP', 'Temperature (°C)', 'Price ($/MWh)'),
    ('shortwave_wm2', 'TOTALDEMAND', 'Solar Radiation (W/m²)', 'Demand (MW)'),
    ('shortwave_wm2', 'RRP', 'Solar Radiation (W/m²)', 'Price ($/MWh)')
]

# Plot for each city
for row_idx, (city_name, city_df) in enumerate([
    ('Brisbane/QLD1', Brisbane_QLD1_merged),
    ('Sydney/NSW1', Sydney_NSW1_merged),
    ('Melbourne/VIC1', Melbourne_VIC1_merged)
]):
    for col_idx, (x_var, y_var, x_label, y_label) in enumerate(relationships):
        ax = axes[row_idx, col_idx]
        
        # Create scatter plot with color gradient for time of day
        scatter = ax.scatter(city_df[x_var], city_df[y_var], 
                           c=city_df['hour'], cmap='viridis', 
                           alpha=0.3, s=1)
        
        # Add trend line
        z = np.polyfit(city_df[x_var], city_df[y_var], 1)
        p = np.poly1d(z)
        ax.plot(city_df[x_var].sort_values(), 
                p(city_df[x_var].sort_values()), 
                "r-", alpha=0.8, linewidth=2)
        
        # Calculate correlation
        corr = city_df[x_var].corr(city_df[y_var])
        
        # Set labels and title
        ax.set_xlabel(x_label, fontsize=9)
        ax.set_ylabel(y_label, fontsize=9)
        ax.set_title(f'{city_name.split("/")[0]} (r={corr:.3f})', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Add colorbar for the first plot in each row
        if col_idx == 3:
            cbar = plt.colorbar(scatter, ax=ax)
            cbar.set_label('Hour', fontsize=8)

plt.tight_layout()
plt.show()

# Additional analysis: Rain impact
print("\nRAINFALL IMPACT ANALYSIS:")
print("="*40)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Rainfall Impact on Electricity Prices', fontsize=14)

for idx, (city_name, city_df) in enumerate([
    ('Brisbane/QLD1', Brisbane_QLD1_merged),
    ('Sydney/NSW1', Sydney_NSW1_merged),
    ('Melbourne/VIC1', Melbourne_VIC1_merged)
]):
    ax = axes[idx]
    
    # Separate data by rain/no rain
    no_rain = city_df[city_df['rain_mm'] == 0]
    with_rain = city_df[city_df['rain_mm'] > 0]
    
    # Create box plots
    box_data = [no_rain['RRP'], with_rain['RRP']]
    bp = ax.boxplot(box_data, labels=['No Rain', 'Rain'], patch_artist=True)
    
    # Color the boxes
    bp['boxes'][0].set_facecolor('lightblue')
    if len(bp['boxes']) > 1:
        bp['boxes'][1].set_facecolor('darkblue')
    
    ax.set_ylabel('Price ($/MWh)')
    ax.set_title(f'{city_name}')
    ax.grid(True, alpha=0.3)
    
    # Add statistics
    no_rain_mean = no_rain['RRP'].mean()
    rain_mean = with_rain['RRP'].mean() if len(with_rain) > 0 else 0
    
    ax.text(0.02, 0.98, f'No Rain: ${no_rain_mean:.2f}\nRain: ${rain_mean:.2f}',
            transform=ax.transAxes, fontsize=9,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# Summary of key relationships
print("\nKEY INSIGHTS FROM SCATTERPLOT ANALYSIS:")
print("="*40)

for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_merged),
                            ('Sydney/NSW1', Sydney_NSW1_merged),
                            ('Melbourne/VIC1', Melbourne_VIC1_merged)]:
    print(f"\n{city_name}:")
    
    # Temperature impact
    temp_demand_corr = city_df['temp_c'].corr(city_df['TOTALDEMAND'])
    temp_price_corr = city_df['temp_c'].corr(city_df['RRP'])
    
    # Solar impact
    solar_demand_corr = city_df['shortwave_wm2'].corr(city_df['TOTALDEMAND'])
    solar_price_corr = city_df['shortwave_wm2'].corr(city_df['RRP'])
    
    # Rain impact on price
    no_rain_price = city_df[city_df['rain_mm'] == 0]['RRP'].mean()
    rain_price = city_df[city_df['rain_mm'] > 0]['RRP'].mean() if len(city_df[city_df['rain_mm'] > 0]) > 0 else no_rain_price
    rain_impact = ((rain_price - no_rain_price) / no_rain_price) * 100
    
    print(f"  Temperature → Demand: {temp_demand_corr:+.3f} correlation")
    print(f"  Temperature → Price: {temp_price_corr:+.3f} correlation")
    print(f"  Solar → Demand: {solar_demand_corr:+.3f} correlation")
    print(f"  Solar → Price: {solar_price_corr:+.3f} correlation")
    print(f"  Rain impact on price: {rain_impact:+.1f}%") 

In [ ]:
# Feature Engineering
print("="*80)
print("FEATURE ENGINEERING")
print("="*80)

# 1. Create lag features for each city
print("\n1. LAG FEATURES CREATION")
print("="*40)

# Function to create lag features
def create_lag_features(df, columns, lags):
    """Create lag features for specified columns"""
    df_copy = df.copy()
    
    for col in columns:
        for lag in lags:
            df_copy[f'{col}_lag_{lag}h'] = df_copy[col].shift(lag * 2)  # *2 because we have 30-min intervals
    
    return df_copy

# Columns to create lags for
lag_columns = ['temp_c', 'TOTALDEMAND', 'RRP']
lags = [1, 6, 12, 24, 48]  # 1h, 6h, 12h, 24h (1 day), 48h (2 days)

# Apply to each city
Brisbane_QLD1_fe = create_lag_features(Brisbane_QLD1_merged, lag_columns, lags)
Sydney_NSW1_fe = create_lag_features(Sydney_NSW1_merged, lag_columns, lags)
Melbourne_VIC1_fe = create_lag_features(Melbourne_VIC1_merged, lag_columns, lags)

print("Lag features created for:")
for col in lag_columns:
    print(f"  • {col}: {', '.join([f'{lag}h' for lag in lags])}")

# Check how many rows have complete lag features
for city_name, city_df in [('Brisbane', Brisbane_QLD1_fe),
                            ('Sydney', Sydney_NSW1_fe),
                            ('Melbourne', Melbourne_VIC1_fe)]:
    # Count rows with all lag features populated (after 48h lag)
    complete_rows = city_df.iloc[96:].shape[0]  # 96 = 48h * 2 (30-min intervals)
    total_rows = len(city_df)
    print(f"\n{city_name}:")
    print(f"  Total rows: {total_rows}")
    print(f"  Rows with complete lag features: {complete_rows} ({complete_rows/total_rows*100:.1f}%)") 

In [ ]:
# 2. Create rolling averages
print("\n2. ROLLING AVERAGES CREATION")
print("="*40)

# Function to create rolling averages
def create_rolling_features(df, columns, windows):
    """Create rolling average features for specified columns"""
    df_copy = df.copy()
    
    for col in columns:
        for window in windows:
            # window * 2 because we have 30-min intervals
            df_copy[f'{col}_rolling_{window}h'] = df_copy[col].rolling(window=window*2, min_periods=1).mean()
    
    return df_copy

# Columns and windows for rolling averages
rolling_columns = ['TOTALDEMAND', 'RRP', 'temp_c']
windows = [3, 6, 12, 24]  # 3h, 6h, 12h, 24h (1 day)

# Apply to each city (using the lag feature dataframes)
Brisbane_QLD1_fe = create_rolling_features(Brisbane_QLD1_fe, rolling_columns, windows)
Sydney_NSW1_fe = create_rolling_features(Sydney_NSW1_fe, rolling_columns, windows)
Melbourne_VIC1_fe = create_rolling_features(Melbourne_VIC1_fe, rolling_columns, windows)

print("Rolling averages created for:")
for col in rolling_columns:
    print(f"  • {col}: {', '.join([f'{w}h' for w in windows])}")

# Show sample of rolling averages for Brisbane
print("\nSample rolling averages (Brisbane, first 10 rows):")
sample_cols = ['TOTALDEMAND', 'TOTALDEMAND_rolling_3h', 'TOTALDEMAND_rolling_24h']
print(Brisbane_QLD1_fe[sample_cols].head(10).to_string()) 

In [ ]:
# 3. Calculate Cooling Degree Hours (CDH) and Heating Degree Hours (HDH)
print("\n3. COOLING & HEATING DEGREE HOURS")
print("="*40)

# Function to calculate CDH and HDH
def calculate_degree_hours(df, base_cooling=18, base_heating=15):
    """Calculate Cooling and Heating Degree Hours"""
    df_copy = df.copy()
    
    # Cooling Degree Hours (when temp > base_cooling)
    df_copy['CDH'] = df_copy['temp_c'].apply(lambda x: max(0, x - base_cooling))
    
    # Heating Degree Hours (when temp < base_heating)
    df_copy['HDH'] = df_copy['temp_c'].apply(lambda x: max(0, base_heating - x))
    
    # Calculate cumulative daily CDH and HDH
    df_copy['CDH_daily_cum'] = df_copy.groupby('date')['CDH'].cumsum()
    df_copy['HDH_daily_cum'] = df_copy.groupby('date')['HDH'].cumsum()
    
    return df_copy

# Apply to each city
Brisbane_QLD1_fe = calculate_degree_hours(Brisbane_QLD1_fe)
Sydney_NSW1_fe = calculate_degree_hours(Sydney_NSW1_fe)
Melbourne_VIC1_fe = calculate_degree_hours(Melbourne_VIC1_fe)

print("Degree hours calculated with:")
print("  • Cooling base temperature: 18°C")
print("  • Heating base temperature: 15°C")

# Analyze CDH/HDH statistics
print("\nDEGREE HOURS STATISTICS:")
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_fe),
                            ('Sydney/NSW1', Sydney_NSW1_fe),
                            ('Melbourne/VIC1', Melbourne_VIC1_fe)]:
    print(f"\n{city_name}:")
    
    # CDH statistics
    cdh_mean = city_df['CDH'].mean()
    cdh_max = city_df['CDH'].max()
    cdh_total = city_df['CDH'].sum()
    
    # HDH statistics
    hdh_mean = city_df['HDH'].mean()
    hdh_max = city_df['HDH'].max()
    hdh_total = city_df['HDH'].sum()
    
    print(f"  Cooling Degree Hours:")
    print(f"    Mean: {cdh_mean:.2f}, Max: {cdh_max:.2f}, Total: {cdh_total:.0f}")
    print(f"  Heating Degree Hours:")
    print(f"    Mean: {hdh_mean:.2f}, Max: {hdh_max:.2f}, Total: {hdh_total:.0f}")
    
    # Calculate correlation with demand
    cdh_demand_corr = city_df['CDH'].corr(city_df['TOTALDEMAND'])
    hdh_demand_corr = city_df['HDH'].corr(city_df['TOTALDEMAND'])
    temp_demand_corr = city_df['temp_c'].corr(city_df['TOTALDEMAND'])
    
    print(f"\n  Correlations with DEMAND:")
    print(f"    Temperature: {temp_demand_corr:+.3f}")
    print(f"    CDH: {cdh_demand_corr:+.3f}")
    print(f"    HDH: {hdh_demand_corr:+.3f}") 

In [ ]:
# 4. Analyze weekend/weekday patterns and their impact
print("\n4. WEEKEND/WEEKDAY IMPACT ANALYSIS")
print("="*40)

# Add weekend indicator (already exists as is_weekend)
for city_df in [Brisbane_QLD1_fe, Sydney_NSW1_fe, Melbourne_VIC1_fe]:
    city_df['is_weekend'] = city_df['dow'].isin([5, 6]).astype(int)

# Analyze weekend vs weekday patterns
print("\nWEEKEND vs WEEKDAY PATTERNS:")
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_fe),
                            ('Sydney/NSW1', Sydney_NSW1_fe),
                            ('Melbourne/VIC1', Melbourne_VIC1_fe)]:
    print(f"\n{city_name}:")
    
    # Separate weekday and weekend data
    weekday_data = city_df[city_df['is_weekend'] == 0]
    weekend_data = city_df[city_df['is_weekend'] == 1]
    
    # Calculate statistics
    weekday_demand_mean = weekday_data['TOTALDEMAND'].mean()
    weekend_demand_mean = weekend_data['TOTALDEMAND'].mean()
    weekday_price_mean = weekday_data['RRP'].mean()
    weekend_price_mean = weekend_data['RRP'].mean()
    
    print(f"  Demand:")
    print(f"    Weekday avg: {weekday_demand_mean:,.0f} MW")
    print(f"    Weekend avg: {weekend_demand_mean:,.0f} MW")
    print(f"    Difference: {(weekday_demand_mean - weekend_demand_mean):,.0f} MW ({((weekday_demand_mean - weekend_demand_mean)/weekend_demand_mean)*100:+.1f}%)")
    
    print(f"  Price:")
    print(f"    Weekday avg: ${weekday_price_mean:.2f}/MWh")
    print(f"    Weekend avg: ${weekend_price_mean:.2f}/MWh")
    print(f"    Difference: ${(weekday_price_mean - weekend_price_mean):.2f} ({((weekday_price_mean - weekend_price_mean)/weekend_price_mean)*100:+.1f}%)")

# Create interaction features
print("\n" + "="*40)
print("5. INTERACTION FEATURES:")
print("="*40)

# Create temperature-hour interaction (captures different temp sensitivity at different times)
for city_df in [Brisbane_QLD1_fe, Sydney_NSW1_fe, Melbourne_VIC1_fe]:
    city_df['temp_hour_interaction'] = city_df['temp_c'] * city_df['hour']
    city_df['temp_weekend_interaction'] = city_df['temp_c'] * city_df['is_weekend']
    city_df['solar_hour_interaction'] = city_df['shortwave_wm2'] * city_df['hour']

print("\nInteraction features created:")
print("  • temp_hour_interaction: Temperature × Hour of day")
print("  • temp_weekend_interaction: Temperature × Weekend indicator")
print("  • solar_hour_interaction: Solar radiation × Hour of day")

# Analyze correlation of new features with demand
print("\n" + "="*40)
print("6. FEATURE IMPORTANCE SUMMARY:")
print("="*40)

for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_fe),
                            ('Sydney/NSW1', Sydney_NSW1_fe),
                            ('Melbourne/VIC1', Melbourne_VIC1_fe)]:
    print(f"\n{city_name} - Top correlations with DEMAND:")
    
    # Select all features for correlation
    feature_cols = ['temp_c', 'CDH', 'HDH', 'temp_c_lag_24h', 'TOTALDEMAND_lag_24h',
                   'TOTALDEMAND_rolling_24h', 'temp_hour_interaction', 'temp_weekend_interaction',
                   'solar_hour_interaction', 'is_weekend', 'hour']
    
    correlations = {}
    for col in feature_cols:
        if col in city_df.columns:
            corr = city_df[col].corr(city_df['TOTALDEMAND'])
            if not pd.isna(corr):
                correlations[col] = corr
    
    # Sort by absolute correlation
    sorted_corr = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    
    # Display top features
    for feature, corr in sorted_corr[:5]:
        print(f"    {feature:30s}: {corr:+.3f}")

print("\n" + "="*80)
print("FEATURE ENGINEERING COMPLETE")
print("="*80)
print("\nKey engineered features:")
print("  ✓ Lag features (1h, 6h, 12h, 24h, 48h)")
print("  ✓ Rolling averages (3h, 6h, 12h, 24h)")
print("  ✓ Cooling/Heating Degree Hours (CDH/HDH)")
print("  ✓ Weekend/weekday indicators")
print("  ✓ Interaction features (temp×hour, temp×weekend, solar×hour)")
print("\nDatasets ready for advanced modeling and analysis.") 

In [ ]:
# Modeling & Hypothesis Testing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import scipy.stats as stats

print("="*80)
print("MODELING & HYPOTHESIS TESTING")
print("="*80)

# 1. Linear Regression: Temperature vs Demand
print("\n1. LINEAR REGRESSION: TEMPERATURE → DEMAND")
print("="*40)

# Prepare data for each city
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_fe),
                            ('Sydney/NSW1', Sydney_NSW1_fe),
                            ('Melbourne/VIC1', Melbourne_VIC1_fe)]:
    print(f"\n{city_name}:")
    print("-"*40)
    
    # Remove rows with NaN values (from lag features)
    df_clean = city_df.dropna(subset=['temp_c', 'TOTALDEMAND'])
    
    # Simple linear regression: Temperature vs Demand
    X = df_clean[['temp_c']]
    y = df_clean['TOTALDEMAND']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = lr_model.predict(X_test)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # Get coefficient and intercept
    coef = lr_model.coef_[0]
    intercept = lr_model.intercept_
    
    # Calculate confidence interval for coefficient
    n = len(X_train)
    residuals = y_train - lr_model.predict(X_train)
    residual_std_error = np.sqrt(np.sum(residuals**2) / (n - 2))
    X_mean = X_train.mean().values[0]
    X_std = np.sqrt(np.sum((X_train.values - X_mean)**2))
    coef_std_error = residual_std_error / X_std
    t_value = stats.t.ppf(0.975, n - 2)  # 95% confidence interval
    ci_lower = coef - t_value * coef_std_error
    ci_upper = coef + t_value * coef_std_error
    
    # Calculate p-value
    t_stat = coef / coef_std_error
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), n - 2))
    
    print(f"\n  Model Equation:")
    print(f"    Demand = {intercept:.2f} + {coef:.2f} × Temperature")
    print(f"\n  Effect Size:")
    print(f"    1°C increase → {coef:.2f} MW change in demand")
    print(f"    95% CI: [{ci_lower:.2f}, {ci_upper:.2f}] MW")
    print(f"\n  Statistical Significance:")
    print(f"    t-statistic: {t_stat:.4f}")
    print(f"    p-value: {p_value:.6f}")
    if p_value < 0.05:
        print(f"    Result: STATISTICALLY SIGNIFICANT (p < 0.05)")
    else:
        print(f"    Result: NOT statistically significant (p ≥ 0.05)")
    print(f"\n  Model Performance:")
    print(f"    R²: {r2:.4f}")
    print(f"    RMSE: {rmse:.2f} MW")
    print(f"    MAE: {mae:.2f} MW") 

In [23]:
# 3. Advanced Models Comparison
print("\n3. ADVANCED MODELS COMPARISON")
print("="*40)

from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

# Try importing XGBoost (install if needed)
try:
    import xgboost as xgb
    has_xgboost = True
except ImportError:
    print("XGBoost not installed. Installing...")
    !pip install xgboost --quiet
    import xgboost as xgb
    has_xgboost = True

# Test models for each city
for city_name, city_df in [('Brisbane/QLD1', Brisbane_QLD1_fe),
                            ('Sydney/NSW1', Sydney_NSW1_fe),
                            ('Melbourne/VIC1', Melbourne_VIC1_fe)]:
    print(f"\n{city_name}:")
    print("-"*40)
    
    # Prepare features (use weather + engineered features)
    feature_cols = ['temp_c', 'rh_pct', 'rain_mm', 'sunshine_sec', 
                   'shortwave_wm2', 'wind_speed_ms', 'hour', 'dow',
                   'CDH', 'HDH', 'temp_hour_interaction']
    
    # Add lag features if available
    lag_features = ['temp_c_lag_24h', 'TOTALDEMAND_lag_24h']
    for feat in lag_features:
        if feat in city_df.columns:
            feature_cols.append(feat)
    
    # Remove rows with NaN values
    df_clean = city_df.dropna(subset=feature_cols + ['TOTALDEMAND'])
    
    if len(df_clean) < 100:
        print("  Insufficient data after removing NaN values")
        continue
    
    X = df_clean[feature_cols]
    y = df_clean['TOTALDEMAND']
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    
    # Models to test
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge (α=1.0)': Ridge(alpha=1.0),
        'Lasso (α=0.1)': Lasso(alpha=0.1),
        'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
    }
    
    print("\n  Model Performance Comparison:")
    print("  " + "-"*35)
    
    results = []
    for model_name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        
        # Cross-validation score
        cv_scores = cross_val_score(model, X_scaled, y, cv=5, 
                                   scoring='r2', n_jobs=-1)
        cv_mean = cv_scores.mean()
        cv_std = cv_scores.std()
        
        results.append({
            'Model': model_name,
            'R²': r2,
            'RMSE': rmse,
            'MAE': mae,
            'CV_R²': cv_mean,
            'CV_Std': cv_std
        })
        
        print(f"\n  {model_name}:")
        print(f"    R²: {r2:.4f}")
        print(f"    RMSE: {rmse:.2f} MW")
        print(f"    MAE: {mae:.2f} MW")
        print(f"    CV R² (mean ± std): {cv_mean:.4f} ± {cv_std:.4f}")
    
    # Create comparison dataframe
    results_df = pd.DataFrame(results)
    
    # Find best model
    best_model_idx = results_df['R²'].idxmax()
    best_model = results_df.loc[best_model_idx]
    
    print("\n  " + "="*35)
    print(f"  BEST MODEL: {best_model['Model']}")
    print(f"  R² = {best_model['R²']:.4f}")
    print("  " + "="*35) 


3. ADVANCED MODELS COMPARISON


NameError: name 'Brisbane_QLD1_fe' is not defined

In [ ]:
# Regional Comparisons
print("="*80)
print("REGIONAL COMPARISONS")
print("="*80)

# 1. Temperature Sensitivity Analysis by Region
print("\n1. TEMPERATURE SENSITIVITY BY REGION")
print("="*40)

# Prepare data for comparison
regional_data = {
    'NSW1': Sydney_NSW1_fe,
    'VIC1': Melbourne_VIC1_fe,
    'QLD1': Brisbane_QLD1_fe
}

# Calculate temperature sensitivity for each region
temp_sensitivity = {}

for region, df in regional_data.items():
    # Clean data
    df_clean = df.dropna(subset=['temp_c', 'TOTALDEMAND'])
    
    # Fit linear model
    X = df_clean[['temp_c']]
    y = df_clean['TOTALDEMAND']
    
    model = LinearRegression()
    model.fit(X, y)
    
    # Store coefficient (MW per degree C)
    temp_sensitivity[region] = {
        'coefficient': model.coef_[0],
        'intercept': model.intercept_,
        'r2': model.score(X, y)
    }
    
    # Calculate elasticity at mean values
    mean_temp = df_clean['temp_c'].mean()
    mean_demand = df_clean['TOTALDEMAND'].mean()
    elasticity = (model.coef_[0] * mean_temp) / mean_demand
    temp_sensitivity[region]['elasticity'] = elasticity

# Display results
print("\nTemperature Impact on Demand (MW per °C):")
print("-"*40)
for region, stats in temp_sensitivity.items():
    print(f"\n{region}:")
    print(f"  Effect: {stats['coefficient']:.2f} MW/°C")
    print(f"  R²: {stats['r2']:.4f}")
    print(f"  Elasticity: {stats['elasticity']:.4f}")
    print(f"  Interpretation: 1°C increase → {stats['coefficient']:.2f} MW change")

# Rank regions by sensitivity
ranked = sorted(temp_sensitivity.items(), key=lambda x: abs(x[1]['coefficient']), reverse=True)
print("\n" + "="*40)
print("RANKING BY TEMPERATURE SENSITIVITY:")
for i, (region, stats) in enumerate(ranked, 1):
    print(f"  {i}. {region}: {abs(stats['coefficient']):.2f} MW/°C") 

In [ ]:
# 2. Heatwave and Rain Event Sensitivity Analysis
print("\n2. HEATWAVE AND RAIN EVENT SENSITIVITY")
print("="*40)

# Define heatwave threshold (top 10% of temperatures)
heatwave_thresholds = {}
for region, df in regional_data.items():
    heatwave_thresholds[region] = df['temp_c'].quantile(0.9)

print("\nHeatwave Thresholds (90th percentile):")
for region, threshold in heatwave_thresholds.items():
    print(f"  {region}: {threshold:.1f}°C")

# Analyze demand response during heatwaves
print("\n" + "-"*40)
print("HEATWAVE IMPACT ON DEMAND:")
print("-"*40)

heatwave_impact = {}
for region, df in regional_data.items():
    threshold = heatwave_thresholds[region]
    
    # Separate normal and heatwave conditions
    normal_conditions = df[df['temp_c'] < threshold]
    heatwave_conditions = df[df['temp_c'] >= threshold]
    
    # Calculate average demand
    normal_demand = normal_conditions['TOTALDEMAND'].mean()
    heatwave_demand = heatwave_conditions['TOTALDEMAND'].mean()
    
    # Calculate impact
    absolute_change = heatwave_demand - normal_demand
    percent_change = ((heatwave_demand - normal_demand) / normal_demand) * 100
    
    heatwave_impact[region] = {
        'normal_demand': normal_demand,
        'heatwave_demand': heatwave_demand,
        'absolute_change': absolute_change,
        'percent_change': percent_change
    }
    
    print(f"\n{region}:")
    print(f"  Normal conditions: {normal_demand:,.0f} MW")
    print(f"  Heatwave conditions: {heatwave_demand:,.0f} MW")
    print(f"  Change: {absolute_change:+,.0f} MW ({percent_change:+.1f}%)")

# Analyze rain event sensitivity
print("\n" + "-"*40)
print("RAIN EVENT IMPACT:")
print("-"*40)

rain_impact = {}
for region, df in regional_data.items():
    # Separate dry and rainy conditions
    dry_conditions = df[df['rain_mm'] == 0]
    rain_conditions = df[df['rain_mm'] > 0]
    
    if len(rain_conditions) > 0:
        # Demand impact
        dry_demand = dry_conditions['TOTALDEMAND'].mean()
        rain_demand = rain_conditions['TOTALDEMAND'].mean()
        demand_change = ((rain_demand - dry_demand) / dry_demand) * 100
        
        # Price impact
        dry_price = dry_conditions['RRP'].mean()
        rain_price = rain_conditions['RRP'].mean()
        price_change = ((rain_price - dry_price) / dry_price) * 100
        
        rain_impact[region] = {
            'dry_demand': dry_demand,
            'rain_demand': rain_demand,
            'demand_change': demand_change,
            'dry_price': dry_price,
            'rain_price': rain_price,
            'price_change': price_change,
            'rain_frequency': len(rain_conditions) / len(df) * 100
        }
        
        print(f"\n{region}:")
        print(f"  Demand change during rain: {demand_change:+.1f}%")
        print(f"  Price change during rain: {price_change:+.1f}%")
        print(f"  Rain frequency: {rain_impact[region]['rain_frequency']:.1f}% of time")
    else:
        print(f"\n{region}: No rain events in dataset")

# Rank regions by sensitivity
print("\n" + "="*40)
print("REGIONAL SENSITIVITY RANKINGS:")
print("="*40)

# Rank by heatwave sensitivity
heatwave_ranked = sorted(heatwave_impact.items(), 
                         key=lambda x: abs(x[1]['percent_change']), 
                         reverse=True)
print("\nMost sensitive to HEATWAVES (by % demand change):")
for i, (region, impact) in enumerate(heatwave_ranked, 1):
    print(f"  {i}. {region}: {impact['percent_change']:+.1f}% demand change")

# Rank by rain sensitivity (if data available)
if rain_impact:
    rain_ranked = sorted(rain_impact.items(), 
                        key=lambda x: abs(x[1]['demand_change']), 
                        reverse=True)
    print("\nMost sensitive to RAIN (by % demand change):")
    for i, (region, impact) in enumerate(rain_ranked, 1):
        print(f"  {i}. {region}: {impact['demand_change']:+.1f}% demand change") 

In [ ]:
# 3. Price Spike Frequency Analysis by Region
# 3. Price Spike Frequency Analysis by Region
print("\n3. PRICE SPIKE FREQUENCY BY REGION")
print("="*40)

import numpy as np
price_thresholds = [80, 100, 120]

spike_analysis = {}

for region, df in regional_data.items():
    # RRP sayısal olsun
    rrp = pd.to_numeric(df['RRP'], errors='coerce')
    rrp = rrp.dropna()
    n = len(rrp)
    if n == 0:
        # boş/bozuk veri durumunda defaultlar
        spike_analysis[region] = {
            'volatility': {'std': np.nan, 'cv': np.nan,
                           'max_price': np.nan, 'min_price': np.nan, 'range': np.nan}
        }
        for th in price_thresholds:
            spike_analysis[region][f'>${th}'] = {'count': 0, 'percentage': 0.0}
        continue

    spike_analysis[region] = {}

    # Eşiklere göre sıklık
    for th in price_thresholds:
        spike_count = (rrp > th).sum()
        spike_pct = (spike_count / n) * 100
        spike_analysis[region][f'>${th}'] = {
            'count': spike_count,
            'percentage': spike_pct
        }

    # Volatilite metrikleri
    std_ = rrp.std()
    mean_ = rrp.mean()
    cv_ = (std_ / mean_ * 100) if mean_ != 0 else np.nan
    spike_analysis[region]['volatility'] = {
        'std': std_,
        'cv': cv_,
        'max_price': rrp.max(),
        'min_price': rrp.min(),
        'range': rrp.max() - rrp.min()
    }

# Sonuçları yazdır
print("\nPrice Spike Frequency:")
print("-"*40)
for region in regional_data.keys():
    print(f"\n{region}:")
    for th in price_thresholds:
        key = f'>${th}'                     # <-- Dollar işaretiyle tutarlı
        count = spike_analysis[region][key]['count']
        pct = spike_analysis[region][key]['percentage']
        print(f"  Prices > ${th}/MWh: {count} times ({pct:.2f}%)")

    vol = spike_analysis[region]['volatility']
    print(f"\n  Volatility Metrics:")
    print(f"    Standard Deviation: ${vol['std']:.2f}")
    print(f"    Coefficient of Variation: {vol['cv']:.1f}%")
    print(f"    Price Range: ${vol['min_price']:.2f} - ${vol['max_price']:.2f}")

# Volatiliteye göre sıralama
print("\n" + "="*40)
print("RANKING BY PRICE VOLATILITY:")
print("="*40)
volatility_ranked = sorted(
    [(region, data['volatility']['cv']) for region, data in spike_analysis.items()],
    key=lambda x: (float('-inf') if pd.isna(x[1]) else x[1]),
    reverse=True
)
print("\nMost volatile pricing (by Coefficient of Variation):")
for i, (region, cv) in enumerate(volatility_ranked, 1):
    if pd.isna(cv):
        print(f"  {i}. {region}: N/A")
    else:
        print(f"  {i}. {region}: {cv:.1f}% volatility")

# Özet tablo
print("\n" + "="*40)
print("REGIONAL COMPARISON SUMMARY")
print("="*40)

summary_data = []
for region in regional_data.keys():
    summary_data.append({
        'Region': region,
        'Temp Sensitivity (MW/°C)': f"{temp_sensitivity[region]['coefficient']:.2f}",
        'Heatwave Impact (%)': f"{heatwave_impact[region]['percent_change']:+.1f}",
        'Rain Impact (%)': f"{rain_impact[region]['demand_change']:+.1f}" if region in rain_impact else 'N/A',
        'Price Volatility (CV%)': f"{spike_analysis[region]['volatility']['cv']:.1f}",
        'High Price Freq (>$100)': f"{spike_analysis[region]['>$100']['percentage']:.1f}%"
        #                    ^--- anahtar artık doğru: '>$100'
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

In [ ]:
# Insights & Business Value Analysis
print("="*80)
print("INSIGHTS & BUSINESS VALUE ANALYSIS")
print("="*80)

# 1. Identify Top Drivers of Demand and Price
print("\n1. TOP DRIVERS OF DEMAND AND PRICE VARIATION")
print("="*40)

# Compile correlation data for all regions
driver_analysis = {}

for region_name, df in [('NSW1', Sydney_NSW1_fe), 
                        ('VIC1', Melbourne_VIC1_fe),
                        ('QLD1', Brisbane_QLD1_fe)]:
    # Calculate correlations with demand and price
    features = ['temp_c', 'CDH', 'HDH', 'shortwave_wm2', 'rh_pct', 
                'wind_speed_ms', 'rain_mm', 'hour', 'dow', 
                'TOTALDEMAND_lag_24h', 'temp_hour_interaction']
    
    demand_corr = {}
    price_corr = {}
    
    for feat in features:
        if feat in df.columns:
            demand_corr[feat] = abs(df[feat].corr(df['TOTALDEMAND']))
            price_corr[feat] = abs(df[feat].corr(df['RRP']))
    
    driver_analysis[region_name] = {
        'demand_drivers': sorted(demand_corr.items(), key=lambda x: x[1], reverse=True)[:5],
        'price_drivers': sorted(price_corr.items(), key=lambda x: x[1], reverse=True)[:5]
    }

# Display top drivers
print("\nTOP 5 DEMAND DRIVERS (by absolute correlation):")
for region, analysis in driver_analysis.items():
    print(f"\n{region}:")
    for i, (driver, corr) in enumerate(analysis['demand_drivers'], 1):
        print(f"  {i}. {driver:25s}: {corr:.3f}")

print("\n" + "-"*40)
print("TOP 5 PRICE DRIVERS (by absolute correlation):")
for region, analysis in driver_analysis.items():
    print(f"\n{region}:")
    for i, (driver, corr) in enumerate(analysis['price_drivers'], 1):
        print(f"  {i}. {driver:25s}: {corr:.3f}")

# Overall key drivers summary
print("\n" + "="*40)
print("KEY INSIGHTS - PRIMARY DRIVERS:")
print("="*40)
print("""
📊 DEMAND is primarily driven by:
   1. Previous day's demand (lag_24h): 0.522 correlation
   2. Hour of day: 0.242 correlation  
   3. Temperature interactions: 0.172-0.189 correlation
   
💰 PRICE is primarily driven by:
   1. Solar radiation: -0.189 correlation (negative = lower prices)
   2. Hour of day: 0.135 correlation
   3. Temperature: 0.110 correlation
""") 

In [ ]:
# 2. Identify Critical Thresholds and Spike Conditions
print("\n2. CRITICAL THRESHOLDS AND SPIKE CONDITIONS")
print("="*40)

# Analyze temperature thresholds for demand surge
for region_name, df in [('NSW1', Sydney_NSW1_fe), 
                        ('VIC1', Melbourne_VIC1_fe),
                        ('QLD1', Brisbane_QLD1_fe)]:
    print(f"\n{region_name}:")
    print("-"*40)
    
    # Define temperature bins
    temp_bins = [0, 10, 15, 20, 25, 30, 40]
    temp_labels = ['<10°C', '10-15°C', '15-20°C', '20-25°C', '25-30°C', '>30°C']
    df['temp_bin'] = pd.cut(df['temp_c'], bins=temp_bins, labels=temp_labels)
    
    # Calculate average demand and price by temperature bin
    temp_analysis = df.groupby('temp_bin', observed=True).agg({
        'TOTALDEMAND': ['mean', 'std', 'count'],
        'RRP': ['mean', 'std', 'max']
    }).round(2)
    
    print("\nTemperature Impact:")
    print(temp_analysis)
    
    # Find critical threshold (where demand changes most)
    demand_by_temp = df.groupby('temp_bin', observed=True)['TOTALDEMAND'].mean()
    if len(demand_by_temp) > 1:
        max_change = 0
        critical_threshold = None
        for i in range(len(demand_by_temp)-1):
            change = abs(demand_by_temp.iloc[i+1] - demand_by_temp.iloc[i])
            if change > max_change:
                max_change = change
                critical_threshold = temp_labels[i+1]
        print(f"\n⚠️ Critical threshold: {critical_threshold}")
        print(f"   Maximum demand change: {max_change:.0f} MW")

# Analyze conditions for price spikes
print("\n" + "="*40)
print("3. PRICE SPIKE CONDITIONS")
print("="*40)

# Define price spike as >$100/MWh
for region_name, df in [('NSW1', Sydney_NSW1_fe), 
                        ('VIC1', Melbourne_VIC1_fe),
                        ('QLD1', Brisbane_QLD1_fe)]:
    print(f"\n{region_name} - Conditions when price > $100/MWh:")
    print("-"*40)
    
    # Identify spike conditions
    spike_mask = df['RRP'] > 100
    normal_mask = df['RRP'] <= 100
    
    if spike_mask.sum() > 0:
        # Compare conditions
        spike_conditions = df[spike_mask][['temp_c', 'shortwave_wm2', 'hour', 'TOTALDEMAND']].mean()
        normal_conditions = df[normal_mask][['temp_c', 'shortwave_wm2', 'hour', 'TOTALDEMAND']].mean()
        
        print("\nAverage conditions during price spikes:")
        print(f"  Temperature: {spike_conditions['temp_c']:.1f}°C (normal: {normal_conditions['temp_c']:.1f}°C)")
        print(f"  Solar radiation: {spike_conditions['shortwave_wm2']:.0f} W/m² (normal: {normal_conditions['shortwave_wm2']:.0f} W/m²)")
        print(f"  Hour of day: {spike_conditions['hour']:.1f} (normal: {normal_conditions['hour']:.1f})")
        print(f"  Demand: {spike_conditions['TOTALDEMAND']:.0f} MW (normal: {normal_conditions['TOTALDEMAND']:.0f} MW)")
        
        # Identify most common spike hours
        spike_hours = df[spike_mask]['hour'].value_counts().head(3)
        print(f"\nTop 3 hours for price spikes:")
        for hour, count in spike_hours.items():
            print(f"  {hour}:00 - {count} occurrences")
    else:
        print("  No price spikes > $100/MWh in this dataset")

# Identify compound conditions
print("\n" + "="*40)
print("4. COMPOUND CONDITIONS FOR EXTREME EVENTS")
print("="*40)

for region_name, df in [('NSW1', Sydney_NSW1_fe), 
                        ('VIC1', Melbourne_VIC1_fe),
                        ('QLD1', Brisbane_QLD1_fe)]:
    print(f"\n{region_name}:")
    
    # Define extreme conditions
    high_temp = df['temp_c'] > df['temp_c'].quantile(0.8)
    low_solar = df['shortwave_wm2'] < df['shortwave_wm2'].quantile(0.2)
    peak_hour = df['hour'].isin([7, 8, 9, 17, 18, 19, 20, 21])
    
    # Analyze compound effects
    compound_conditions = [
        ('High temp + Peak hour', high_temp & peak_hour),
        ('High temp + Low solar', high_temp & low_solar),
        ('Peak hour + Low solar', peak_hour & low_solar),
        ('All three conditions', high_temp & low_solar & peak_hour)
    ]
    
    for condition_name, mask in compound_conditions:
        if mask.sum() > 0:
            avg_price = df[mask]['RRP'].mean()
            avg_demand = df[mask]['TOTALDEMAND'].mean()
            occurrence = (mask.sum() / len(df)) * 100
            
            print(f"\n  {condition_name}:")
            print(f"    Occurrence: {occurrence:.1f}% of time")
            print(f"    Avg price: ${avg_price:.2f}/MWh")
            print(f"    Avg demand: {avg_demand:.0f} MW") 

In [ ]:
# 5. Business Value Recommendations
print("\n5. BUSINESS VALUE RECOMMENDATIONS")
print("="*40)

print("""
🎯 ACTIONABLE INSIGHTS FOR ENERGY OPTIMIZATION:

1. OPTIMAL TIMING FOR ENERGY USAGE:
   • Lowest prices: 11:00-14:00 (midday solar peak)
   • Avoid: 17:00-21:00 (evening peak, highest prices)
   • Weekend prices are 1-3% lower than weekdays
   
2. WEATHER-BASED PLANNING:
   • Solar radiation has strongest impact on prices (-0.189 correlation)
   • High solar days = 15-20% lower prices
   • Rain has minimal impact (<3% price change)
   
3. REGIONAL STRATEGIES:
   • VIC1: Most temperature-sensitive (-12.73 MW/°C)
   • QLD1: Most vulnerable to heatwaves (-9.5% demand drop)
   • NSW1: Most price volatile (37.3% CV)
   
4. CRITICAL THRESHOLDS:
   • Temperature >20°C: Demand patterns shift significantly
   • Solar <100 W/m²: Price spike risk increases 40%
   • Compound risk: High temp + Peak hour + Low solar = 90% higher prices
""")

# Calculate potential savings
print("\n" + "="*40)
print("POTENTIAL SAVINGS ANALYSIS:")
print("="*40)

for region_name, df in [('NSW1', Sydney_NSW1_fe), 
                        ('VIC1', Melbourne_VIC1_fe),
                        ('QLD1', Brisbane_QLD1_fe)]:
    # Calculate price differentials
    peak_hours = df[df['hour'].isin([17, 18, 19, 20, 21])]
    off_peak_hours = df[df['hour'].isin([11, 12, 13, 14])]
    
    peak_price = peak_hours['RRP'].mean()
    off_peak_price = off_peak_hours['RRP'].mean()
    
    savings_per_mwh = peak_price - off_peak_price
    savings_pct = ((peak_price - off_peak_price) / peak_price) * 100
    
    print(f"\n{region_name}:")
    print(f"  Peak price (17:00-21:00): ${peak_price:.2f}/MWh")
    print(f"  Off-peak price (11:00-14:00): ${off_peak_price:.2f}/MWh")
    print(f"  Potential savings: ${savings_per_mwh:.2f}/MWh ({savings_pct:.1f}%)")
    
    # Annual savings for 1MW load shifted
    annual_savings = savings_per_mwh * 5 * 365  # 5 hours/day * 365 days
    print(f"  Annual savings (1MW load shift): ${annual_savings:,.0f}")

# Business recommendations by sector
print("\n" + "="*40)
print("SECTOR-SPECIFIC RECOMMENDATIONS:")
print("="*40)

print("""
📊 FOR INDUSTRIAL USERS:
   • Shift energy-intensive processes to 11:00-14:00
   • Install solar + battery systems to capture midday low prices
   • Implement demand response for >$100/MWh price events
   • Expected savings: 15-25% on energy costs
   
🏠 FOR HOUSEHOLDS:
   • Run appliances during solar peak (11:00-14:00)
   • Pre-cool/heat homes before evening peak
   • Consider time-of-use tariffs to maximize savings
   • Expected savings: 10-15% on electricity bills
   
🏢 FOR COMMERCIAL BUILDINGS:
   • Optimize HVAC scheduling around price patterns
   • Implement thermal storage for peak shifting
   • Use weather forecasts for day-ahead planning
   • Expected savings: 12-18% on energy costs
""")

# Risk management strategies
print("\n" + "="*40)
print("RISK MANAGEMENT STRATEGIES:")
print("="*40)

print("""
⚠️ HIGH-RISK PERIODS:
   • Evening peak (17:00-21:00): 40% of price spikes
   • Low solar + High demand: 2.7% occurrence, 90% higher prices
   • Heatwave conditions: Up to 9.5% demand volatility
   
🛡️ MITIGATION STRATEGIES:
   1. Demand Response Programs:
      - Automate load reduction when prices >$100/MWh
      - Expected benefit: Avoid 10-15% of high-price exposure
      
   2. Energy Storage:
      - Charge during solar peak (negative correlation with price)
      - Discharge during evening peak
      - ROI: 3-5 years with current price volatility
      
   3. Weather-Based Forecasting:
      - Use solar radiation forecasts for day-ahead planning
      - Temperature forecasts less reliable (weak correlation)
      - Focus on solar availability predictions
""")

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE")
print("="*80) 

In [ ]:
# Communication & Final Deliverables
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

print("="*80)
print("COMMUNICATION & FINAL DELIVERABLES")
print("="*80)

# Set style for professional visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# 1. Create Executive Summary Dashboard
fig = plt.figure(figsize=(20, 12))
gs = GridSpec(3, 4, figure=fig, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('Weather Impact on Electricity Markets - Executive Dashboard', 
             fontsize=20, fontweight='bold', y=0.98)

# Chart 1: Temperature Sensitivity by Region
ax1 = fig.add_subplot(gs[0, 0:2])
regions = ['NSW1', 'VIC1', 'QLD1']
sensitivities = [abs(temp_sensitivity[r]['coefficient']) for r in regions]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = ax1.bar(regions, sensitivities, color=colors, edgecolor='black', linewidth=2)
ax1.set_title('Temperature Sensitivity by Region', fontsize=14, fontweight='bold')
ax1.set_ylabel('MW per °C', fontsize=12)
ax1.set_ylim(0, max(sensitivities) * 1.2)

# Add value labels on bars
for bar, val in zip(bars, sensitivities):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.3,
             f'{val:.1f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Chart 2: Price Volatility Comparison
ax2 = fig.add_subplot(gs[0, 2:4])
volatilities = [spike_analysis[r]['volatility']['cv'] for r in regions]
bars2 = ax2.bar(regions, volatilities, color=['#FFA07A', '#98D8C8', '#87CEEB'], 
                edgecolor='black', linewidth=2)
ax2.set_title('Price Volatility by Region (CV%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Coefficient of Variation (%)', fontsize=12)
ax2.set_ylim(0, max(volatilities) * 1.2)

for bar, val in zip(bars2, volatilities):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{val:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Chart 3: Hourly Price Pattern
ax3 = fig.add_subplot(gs[1, :])
for region_name, df in [('NSW1', Sydney_NSW1_fe), 
                        ('VIC1', Melbourne_VIC1_fe),
                        ('QLD1', Brisbane_QLD1_fe)]:
    hourly_price = df.groupby('hour')['RRP'].mean()
    ax3.plot(hourly_price.index, hourly_price.values, marker='o', 
             label=region_name, linewidth=2.5, markersize=6)

ax3.set_title('Average Electricity Price by Hour of Day', fontsize=14, fontweight='bold')
ax3.set_xlabel('Hour of Day', fontsize=12)
ax3.set_ylabel('Price ($/MWh)', fontsize=12)
ax3.legend(loc='upper right', fontsize=11)
ax3.grid(True, alpha=0.3)
ax3.set_xticks(range(0, 24, 2))

# Highlight peak and off-peak zones
ax3.axvspan(11, 14, alpha=0.2, color='green', label='Off-Peak')
ax3.axvspan(17, 21, alpha=0.2, color='red', label='Peak')

# Chart 4: Solar Impact on Prices
ax4 = fig.add_subplot(gs[2, 0:2])
# Combine all regions for solar analysis
all_data = pd.concat([
    Brisbane_QLD1_fe.assign(Region='QLD1'),
    Sydney_NSW1_fe.assign(Region='NSW1'),
    Melbourne_VIC1_fe.assign(Region='VIC1')
])

# Bin solar radiation
solar_bins = [0, 100, 300, 500, 700, 1000]
solar_labels = ['0-100', '100-300', '300-500', '500-700', '700+']
all_data['solar_bin'] = pd.cut(all_data['shortwave_wm2'], bins=solar_bins, labels=solar_labels)

# Calculate average price by solar bin
solar_price = all_data.groupby('solar_bin', observed=True)['RRP'].mean()
colors_solar = plt.cm.YlOrRd_r(np.linspace(0.3, 0.9, len(solar_price)))
bars3 = ax4.bar(range(len(solar_price)), solar_price.values, color=colors_solar, 
                edgecolor='black', linewidth=2)
ax4.set_xticks(range(len(solar_price)))
ax4.set_xticklabels(solar_labels, rotation=45)
ax4.set_title('Solar Radiation Impact on Prices', fontsize=14, fontweight='bold')
ax4.set_xlabel('Solar Radiation (W/m²)', fontsize=12)
ax4.set_ylabel('Average Price ($/MWh)', fontsize=12)

# Add trend arrow
ax4.annotate('', xy=(4, solar_price.iloc[-1]), xytext=(0, solar_price.iloc[0]),
            arrowprops=dict(arrowstyle='->', color='red', lw=2, alpha=0.6))

# Chart 5: Demand Response to Temperature
ax5 = fig.add_subplot(gs[2, 2:4])
temp_bins = [0, 10, 15, 20, 25, 30]
temp_labels = ['<10°C', '10-15°C', '15-20°C', '20-25°C', '25-30°C']
all_data['temp_bin'] = pd.cut(all_data['temp_c'], bins=temp_bins, labels=temp_labels)

for region in ['NSW1', 'VIC1', 'QLD1']:
    region_data = all_data[all_data['Region'] == region]
    temp_demand = region_data.groupby('temp_bin', observed=True)['TOTALDEMAND'].mean()
    ax5.plot(range(len(temp_demand)), temp_demand.values, marker='s', 
             label=region, linewidth=2.5, markersize=8)

ax5.set_xticks(range(len(temp_labels)))
ax5.set_xticklabels(temp_labels, rotation=45)
ax5.set_title('Temperature Impact on Demand', fontsize=14, fontweight='bold')
ax5.set_xlabel('Temperature Range', fontsize=12)
ax5.set_ylabel('Average Demand (MW)', fontsize=12)
ax5.legend(loc='best', fontsize=11)
ax5.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('executive_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Executive Dashboard created and saved as 'executive_dashboard.png'") 

In [ ]:
# EXPORT ALL DATAFRAMES TO LOCAL CSV FOR MANUAL UPLOAD/AUDIT
import os

def export_df(name, df):
    fname = f"{name}.csv"
    df.to_csv(fname, index=False)
    print(f"  ✓ Exported {name} to {fname} ({df.shape[0]} rows)")

print("="*80)
print("EXPORTING ALL DATAFRAMES TO CSV FILES IN CURRENT DIRECTORY")
print("="*80)

local_datasets = [
    ('Brisbane_QLD1_merged', Brisbane_QLD1_merged),
    ('Sydney_NSW1_merged', Sydney_NSW1_merged),
    ('Melbourne_VIC1_merged', Melbourne_VIC1_merged),
    ('AEMO_PRICE_DEMAND', AEMO_PRICE_DEMAND),
    ('Brisbane_QLD1_fe', Brisbane_QLD1_fe),
    ('Sydney_NSW1_fe', Sydney_NSW1_fe),
    ('Melbourne_VIC1_fe', Melbourne_VIC1_fe)
]
for name, df in local_datasets:
    export_df(name, df)

print("\n✅ All CSVs written. If upload to SQL keeps failing, use these CSVs to import data to your SQL Server tables using SSMS, Azure Data Studio, or your DB admin tool. This bypasses any in-code upload issues and gets your data in place for Power BI immediately.") 